In [1]:
from torchvision.models import densenet161
from torch import nn
import numpy as np
from PIL import Image
from utilmaps import *
import os, sys, cv2, torch, json, shutil
from matplotlib import cm
import scipy as sp
import scipy.ndimage
from util_storage import *

    
from scipy.ndimage import label, generate_binary_structure


In [2]:
dataset_folder = './dataset/'

In [3]:
def loadMask(path, size_limit=None):
    with Image.open(path) as im:
        mask = np.array(im.convert('1')).astype(bool)
        
        if __name__=='__main__':
            fig, axs = plt.subplots(1, 2)
            axs[0].imshow(mask)
            
        if size_limit is not None:
            s = generate_binary_structure(2,2)
            label_structures, num_features = label(mask, s)

            for connected_component_id in [x+1 for x in range(num_features)]:
                current_component = label_structures == connected_component_id

                if np.nansum(current_component) > size_limit:
                    mask[current_component] = False
    return mask

def assignLabel(mapi,mask):
    results = []
    
    s = generate_binary_structure(2,2)
    label_structures, num_features = label(mask, s)

    for connected_component_id in [x+1 for x in range(num_features)]:
        current_component = label_structures == connected_component_id
       
        current_component_map = current_component * mapi
        
        # Per pixel confidence
        proportion = float(np.nansum(current_component_map)/ np.nansum(current_component))
        results.append((int(np.nansum(current_component)),proportion))
    return results

def generateMap(image_name, image_mask):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    jet_cmap = cm.get_cmap('jet')

    try:
        image_original =  np.array(Image.open(image_name))[:,:,0].astype(float)/255
    except:
        image_original =  np.array(Image.open(image_name)).astype(float)/255

    image_mask = np.array(Image.open(image_mask).convert('L')).astype(bool)    
    
    # FASE 1
    model = densenet161()
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    model.load_state_dict(torch.load('model_64.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    builder = MapBuilder(model,64,55,model_input_size=224,batch_size=500)

    roi_mask_beforeth = builder.generateMap(image_original,image_mask)*image_mask
    
    th = 0.5
 
    roi_mask = roi_mask_beforeth > th
    builder.batch_size = 4000
    del model
    torch.cuda.empty_cache()
    
    # FASE 2
    model = densenet161()
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    model.load_state_dict(torch.load('model_30.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    
    builder = MapBuilder(model,30,25,model_input_size=64)
    microcyst_map = np.nan_to_num(builder.generateMap(image_original,roi_mask))
    microcyst_map = sp.ndimage.filters.gaussian_filter(microcyst_map, 3, mode='constant')
     
    return roi_mask_beforeth, microcyst_map


In [4]:
images_path = dataset_folder + 'ORIGINALS/'
masks_path = dataset_folder + 'MASKS/'
microcysts_path = dataset_folder + 'MICROCYSTS/'

images_names = list(os.listdir(images_path))
masks_names = list(os.listdir(masks_path))
microcysts_names = list(os.listdir(microcysts_path))

all_folds = loadObj('FOLD_DATA_FN_L_6V2T1')

In [5]:
fold_stage = 'test'

labels_macro = []
labels_micro = []
labels_micro_faux = []


i=0.0
for fold_data in all_folds:
    print('{}%'.format(i/len(all_folds)))
    i+=1
    for image_name in fold_data[fold_stage]:
            image_name = image_name.replace('dataset/','').replace('.npy','').replace('.png','').replace('png','.png').replace('tiff','.tiff').replace('tif','.tif').replace('bmp','.bmp').replace('jpg','.jpg').replace('jpeg','.jpeg')
            image_path = images_path + image_name
            mask_path = masks_path + image_name.replace('.','')+'.png'
            microcyst_path = microcysts_path + image_name.replace('.','')+'.png'

            map_stage1, map_stage2 = generateMap(image_path, mask_path)

            # STAGE 1
            fullmask = loadMask(microcyst_path)
            labels_macro += assignLabel(map_stage1,fullmask)

            # STAGE 2
            micromask = loadMask(microcyst_path, size_limit = 900)
            labels_micro += assignLabel(map_stage2,micromask)
            
            # STAGE 2 FAUX
            labels_micro_faux += assignLabel(map_stage2,fullmask)

            saveObj(labels_macro,'labels_macro')
            saveObj(labels_micro,'labels_micro')
            saveObj(labels_micro_faux,'labels_micro_faux')

            if False:
                print(labels_macro)
                print(labels_micro)
                print(np.nanmax(map_stage1))
                print(np.nanmin(map_stage1))
                print(np.nanmax(map_stage2))
                print(np.nanmin(map_stage2))

                plt.imshow(map_stage1)
                plt.show()
                plt.imshow(map_stage2)
                plt.show()

                plt.imshow(Image.open(image_path))
                plt.show()
                plt.imshow(Image.open(microcyst_path))
                plt.show()
                print(image_path)

                print(microcyst_path)
                


0.0%



KeyboardInterrupt



Error in callback <function flush_figures at 0x7f380188cee0> (for post_execute):



KeyboardInterrupt

